# Lambda functions

Lambda (or anonymous) functions were introduced in C++11 and enhanced in C++14. They can be defined on-the-fly anywhere in the code where they are needed. This avoid a previous declaration of the function, useless when the function is to be used only once. Moreover, the capture of local variables avoids the tedious definition of an object-function (whose `operator()` is overloaded). In simple cases, the resulting code is more concise and clearer.

## One-time use, anonymous, functions

Some ordinary functions are meant to be used only once. Since nested functions are not allowed in C++, one must pollute the global namespace:

In [ ]:
#include <vector>

In [ ]:
int reduce( std::vector<int> collection, int accumulator, int (*op)( int, int ) ) {
  for ( int element : collection ) {
    accumulator = op(accumulator,element) ;
  }
  return accumulator ;
}

In [ ]:
int add( int val1, int val2 ) { return (val1+val2) ; }

In [ ]:
int multiply( int val1, int val2 ) { return (val1*val2) ; }

In [ ]:
#include <iostream>

In [ ]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
std::cout<<reduce(numbers,0,add)<<std::endl ;
std::cout<<reduce(numbers,1,multiply)<<std::endl ;

C++11 allows one-the-fly definition of an anonymous function, where it is to be used. **The function name is replaced with `[]`**. It is called a *lambda*:

In [ ]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
std::cout<<reduce(numbers,0,[](int i1, int i2){ return i1+i2 ; })<<std::endl ;
std::cout<<reduce(numbers,1,[](int i1, int i2){ return i1*i2 ; })<<std::endl ;

## Return type

As above, the compiler can guess the return type of your lambda by inspecting the instructions `return`. To improve the readability of the code, or to help the compiler in certain ambiguous cases, we can explain the return type of a lambda. This is called **trailing return type declaration**.

In [ ]:
int addition = reduce( numbers, 0, [](int i1, int i2) -> int { return i1+i2 ; } ) ;
std::cout << addition << std::endl ;

## To modify elements, pass the argument by reference

If your lambda must modify the received element, you have to (naturally) declare this element as a reference:

In [ ]:
std::vector<int> v { 1, 2, 3, 4, 5 } ;

std::for_each(v.begin(),v.end(),[]( int & i ){
  i = 2*i ;
}) ;

std::for_each(v.begin(),v.end(),[]( int i ){
  std::cout<<i<<' ' ;
}) ;
std::cout<<std::endl ;

## Capturing local variables

Like any ordinary function, it only sees its arguments, local variables and global variables from the file. This does not compile:

In [ ]:
%%file tmp.capture.cpp

#include <iostream>
#include <vector>
#include <algorithm>

int main() {
    
  std::vector<int> numbers { 1, 2, 3, 4, 5 } ;
  int coef { 3 } ;
  std::for_each(
    numbers.begin(),numbers.end(),
    []( int a_n ){ std::cout << (coef*a_n) << " " ; }
  ) ;
  
}

In [ ]:
!rm -f tmp.capture.exe && g++  -std=c++17 tmp.capture.cpp -o tmp.capture.exe

A lambda function can include between its initial brackets a list of variables to be "captured" from its context, by value or by reference:
* `[]`: no variable;
* `[x,y,&j]`: `x` and` y` by value and `j` by reference;
* `[&]`: all variables by reference;
* `[=]`: all variables by copy;
* `[=,&j]`: all variables by copy except `j` by reference;
* `[&,j]`: all variables by reference, except `j` by copy.

Capturing `coef` allows to resolve the previous problem:

In [ ]:
std::vector<int> numbers { 1, 2, 3, 4, 5 } ;
int coef = 3 ;
std::for_each(
  numbers.begin(),numbers.end(),
  [coef]( int a_n ){ std::cout << (coef*a_n) << " " ; }
) ;

A lambda is equivalent to some function-object, which capture the variables as members, and reuse them in the implementation of `operator()`:

In [ ]:
class Multiplier
 {
  public :
    Multiplier( int a_coef ) : m_coef(a_coef) {}
    void operator() ( int a_n ) { std::cout << (m_coef*a_n) << " " ; }
  private :
    int const m_coef ;
 } ;

In [ ]:
std::vector<int> numbers { 1, 2, 3, 4, 5 } ;
Multiplier m { 3 } ;
std::for_each(numbers.begin(),numbers.end(),m) ;

## Capturing by reference

In [ ]:
std::vector<int> v {1,2,3,4,5} ;
    
int accumulator = 0 ;
std::for_each(v.begin(), v.end(), [&accumulator]( int i ){
  accumulator += i ;
}) ;
    
std::cout<<accumulator<<std::endl ;

**BEWARE**: when capturing by reference, as with any reference, the behavior is undefined if the original variable disappears before the lambda function is used.

## Storing and reusing lambdas

In [ ]:
#include <vector>
#include <algorithm>
#include <iostream>

A lambda function is a "first class object", and can be stored in a variable, to be reused later as any normal function. **The type of the lambda is implementation-dependent**. The usual practice is to declare above variable `auto`.

In [ ]:
auto constexpr mult2 = []( int i ){ std::cout<<(2*i)<<" " ; } ;

std::vector<int> v1 {1,2,3,4,5} ;
std::for_each(v1.begin(), v1.end(), mult2) ;
std::cout<<std::endl ;

std::vector<int> v2 {6,7,8} ;
std::for_each(v2.begin(), v2.end(), mult2) ;
std::cout<<std::endl ;

A noteworthy difference with an ordinary function : you can **nest it** in any block.

In [ ]:
void process( std::vector<int> const & v ) {
  auto constexpr mult2 = []( int i ){ std::cout<<(2*i)<<" " ; } ;
  std::for_each(v.begin(), v.end(), mult2) ;
  std::cout<<std::endl ;
}

std::vector<int> v1 {1,2,3,4,5} ;
process(v1) ;

std::vector<int> v2 {6,7,8} ;
process(v2) ;

Again, beware not to capture by reference something which may be destructed before the lambda is used.

In [ ]:
#include <vector>
#include <algorithm>
#include <iostream>

In [ ]:
int coef ;
auto mult = [&coef]( int i ){ std::cout<<(coef*i)<<" " ; } ;
std::vector<int> v {1,2,3,4,5} ;
  
coef = 2 ;  
std::for_each(v.begin(), v.end(), mult) ;
std::cout<<std::endl ;

coef = 3 ;  
std::for_each(v.begin(), v.end(), mult) ;
std::cout<<std::endl ;

## Generic lambdas

If you want to reuse your lambda with different input types, you can also use `auto` in the functions parameters:

In [ ]:
auto print = []( auto val ){ std::cout<<val<<' ' ; } ;

std::vector<int> vi{ 1, 2, 3, 4, 5 } ;
std::for_each(vi.begin(),vi.end(),print) ;
std::cout<<std::endl ;

std::vector<double> vd{ 1.1, 2.2, 3.3, 4.4 } ;
std::for_each(vd.begin(),vd.end(),print) ; 
std::cout<<std::endl ;

The first `auto` triggers type inference. The second is rather some simplified form of `template`. If we look for the equivalent function-object, it might look like this:

In [ ]:
class Print
 {
  public :
    template< typename Value >
    void operator()( Value val )
     { std::cout<<val<<' ' ; }
 } ;

Note that it is the execution operator (`operator()`) that is parameterized, and not the class itself.

# Take away

- a lambda function is anonymous, and usually meant to be used once ;
- yet one can store it in a variable, which makes it some kind of nested function ;
- thanks to the capture, lambda functions are a concise version of object-functions ;
- a generic lambda is equivalent to an object-function with a template operator().

# Questions ?

# Exercise

Replace below `random_unit` and `Pow` with lambdas functions. Make sure that you always get the same end result throughout your trials.

In [ ]:
%%file tmp.lambdas.cpp

#include <vector>
#include <algorithm>
#include <numeric>
#include <iostream>
#include <cmath>

// random double value in [-1,1]
void random_unit( double & a_value )
 { a_value = ((2.*std::rand())/RAND_MAX-1.) ; }

// compute value^degree
struct Pow
 {
  int m_degree ;
  Pow( int a_degree ) : m_degree {a_degree} {}
  double operator()( double a_value ) const
   { return std::pow(a_value,m_degree) ; }
 } ;

// main program
int main()
 {
  int const DIM {10} ;
  int const DEGREE {5} ;
  
  // generate random input
  std::vector<double> input(DIM) ;
  std::for_each(input.begin(),input.end(),random_unit) ;

  // compute output
  std::vector<double> output(DIM) ;
  std::transform(input.begin(),input.end(),output.begin(),Pow(DEGREE)) ;
  
  // print sum
  std::cout<<std::accumulate(output.begin(),output.end(),0.)<<std::endl ;
}

In [ ]:
!rm -f tmp.lambdas.exe && g++  -std=c++17 tmp.lambdas.cpp -o tmp.lambdas.exe

In [ ]:
!./tmp.lambdas.exe

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*